## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Étiquetage morphosyntaxique (POS Tagging) 
- (Lemmatisation)
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Extraction de collocations significatives (en fonction du Log-likelihood ratio)
- Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt
- Extraction de termes MeSH et SNOMED présents dans les données

### **Lire le corpus** 

In [254]:
lng = 'fr'
acteur = 'msss'
tag = 'dependances'
if tag:
    file = acteur + '/' + acteur + '_' + tag + '.csv'

else:
    if lng == 'fr':
        file = acteur +'.csv'
    if lng == 'en':
        file = acteur + '_en.csv'

In [255]:
import shutil, re, random
from os import listdir, chdir, path
from pathlib import Path
from pandas import *

import nltk
#nltk.download(['popular'])
from nltk.tokenize import RegexpTokenizer
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")
from nltk import bigrams, trigrams, ngrams, everygrams
from nltk.probability import FreqDist


import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG=lng)


from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import binom, chi2

In [256]:
def lire_corpus(csv_file, langue=lng):
    base_path = '../03-corpus/2-data/'
    file_path = path.join(base_path, '1-' + langue, csv_file)

    #encoding='ISO 8859-1'
    with open(file_path, "r", encoding='utf-8') as f:
        data = read_csv(file_path, encoding='utf-8', sep=',')
        data = data[~data["url"].str.contains('pdf')] #  Problèmes 
        data = data['text']
    return data

In [257]:
data = lire_corpus(file) 
nb_docs = len(data)
print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 165 documents.


In [258]:
data

0      Dépendances (alcool, drogues, jeu) Campagne Ca...
1      Volet québécois du Programme sur l'usage et le...
2      Dépendances Dépendances (alcool, drogues, jeu)...
3      Dépendances (alcool, drogues, jeu) Campagne Ca...
4      Dépendances (alcool, drogues, jeu) Campagne su...
                             ...                        
160    [ Aller au contenu ] [ Aller a la navigation ]...
161    [ Aller au contenu ] [ Aller a la navigation ]...
162    [ Aller au contenu ] [ Aller a la navigation ]...
163    [ Aller au contenu ] [ Aller a la navigation ]...
164    [ Aller au contenu ] [ Aller a la navigation ]...
Name: text, Length: 165, dtype: object

### **Nettoyage**

In [259]:
punct = '[!#$%&•►*+,;\/\\<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'
postals = '([a-zA-Z]+\d+|\d+[a-zA-Z]+)+'
# phones = '\d{3}\s\d{3}-\d{4}' #très simple (trop)

text = [str(t).strip('\n').lower().replace('’', '\'') for t in data]
text = [re.sub(spaces, ' ', t) for t in text]
text = [re.sub(postals, ' STOP ', t) for t in text]
text = [re.sub(punct, ' STOP ', t) for t in text]
text = [t.replace("  ", " " ) for t in text]

In [260]:
nb_docs = len(text)
print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 165 documents.


### **Extraire un échantillon aléatoire**

Sinon, on n'arrive pas à traiter la totalité du corpus pour des raisons de performance

In [261]:
def sample_corpus(corpus, ratio):
    n = round(ratio * len(corpus))
    corpus = random.sample(text, n)
    print("On va travailler sur un échantillon correspondant à environ " + str(ratio * 100) + " % des documents du corpus, soit {} documents". format(len(corpus)))
    return " ".join(corpus)
    
corpus = sample_corpus(text, 1)

On va travailler sur un échantillon correspondant à environ 100 % des documents du corpus, soit 165 documents


### **Filtrage (MWE - stopwords formés de plusieurs tokens)**
Surtout pour filtrer les expressions relatives à l'architecture d'information / navigation Web

In [262]:
def filter_mwesw(corpus):
    file_mwesw = '../04-filtrage/mwe_stopwords.txt'
    with open (file_mwesw, 'r', encoding='utf-8') as f:
        mwe_sw = [t.lower().strip('\n') for t in f.readlines()]
    for mwe in mwe_sw:
        corpus = corpus.replace(mwe, ' STOP ').replace('  ', " ")
    return corpus

In [263]:
corpus = filter_mwesw(corpus)

### **Tokenisation / POS tagging** (TreeTagger)  
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [264]:
# Ici, on tokenise une première fois avec le Regex Tokenizer de NLTK pour voir combien de temps ça devrait 
# prendre au Tree Tagger pour tokeniser et tagger notre corpus
def tok(corpus):
    # Seulement les caractères alphabétiques
    tokens = tokenizer_re.tokenize(corpus)
    print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len(tokens)))
    temps = round(len(tokens) / 15000 / 60)
    print('Le POS tagging devrait prendre environ {} minutes.'.format(temps))

temps = tok(corpus)

Avec le RegExpTokenizer, notre corpus contient 30677 tokens.
Le POS tagging devrait prendre environ 0 minutes.


In [265]:
def tagging(corpus):
    output = []
    for t in tagger.tag_text(corpus):
        try: 
            output.append((t.split('\t')[0], t.split('\t')[1]))
        except Exception as e:
            output.append(('STOP', 'NAM'))

    return output


In [266]:
tagged = tagging(corpus)
tokens = [t[0] for t in tagged]

In [267]:
tagged

[('msss', 'NOM'),
 ('répertoires', 'NOM'),
 ('ressources', 'NOM'),
 ('en', 'PRP'),
 ('dépendances', 'NOM'),
 ('05', 'NUM'),
 ('-', 'PUN'),
 ('estrie', 'NOM'),
 ('fiche', 'NOM'),
 ('de', 'PRP'),
 ('la', 'DET:ART'),
 ('ressource', 'NOM'),
 ('répertoire', 'NOM'),
 ('des', 'PRP:det'),
 ('ressources', 'NOM'),
 ('en', 'PRP'),
 ('dépendances', 'NOM'),
 ('STOP', 'NOM'),
 ('STOP', 'INT'),
 ('STOP', 'NOM'),
 ('des', 'PRP:det'),
 ('ressources', 'NOM'),
 ('de', 'PRP'),
 ('la', 'DET:ART'),
 ('région', 'NOM'),
 ('05', 'NUM'),
 ('région', 'NOM'),
 ('05', 'NUM'),
 ('-', 'PUN'),
 ('estrie', 'NOM'),
 ('centre', 'NOM'),
 ("l'", 'DET:ART'),
 ('envolée', 'NOM'),
 ('de', 'PRP'),
 ('granby', 'NOM'),
 ('350', 'NUM'),
 ('STOP', 'NOM'),
 ('chemin', 'NOM'),
 ('ostiguy', 'ADJ'),
 ('shefford', 'NOM'),
 ('(', 'PUN'),
 ('québec', 'NOM'),
 (')', 'PUN'),
 ('STOP', 'NOM'),
 ('m', 'NOM'),
 ('STOP', 'NOM'),
 ('7', 'NUM'),
 ('téléphone', 'NOM'),
 (':', 'PUN'),
 ('450', 'NUM'),
 ('378-5326', 'NUM'),
 ('télécopieur', 'NOM')

In [268]:
# Si on veut lemmatiser
### **Mapping POS Tags** (FRMG)

#Pour utiliser adéquatement notre lemmatiseur par la suite (FrenchLefffLemmatizer), on va mapper les étiquettes morphosyntaxiques du TreeTagger à celles que prend le lemmatiseur (celles issues de FRMG)

#http://alpage.inria.fr/frmgwiki/content/tagset-frmg

file_path = '../04-filtrage/mapping_treeTagger_lefff.csv'

with open(file_path) as f:
   csv = read_csv(f)

treeTag = [term for term in csv['TreeTagger'].tolist()] 
lefff = [term for term in csv['Lefff'].tolist()]

mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

In [269]:
#tagged_lem = [[t.split('\t')[0], mapping[t.split('\t')[1]]] for t in tagger.tag_text(corpus)]

In [270]:
# Si on veut lemmatiser
### **Lemmatisation** (FrenchLefffLemmatizer)
#https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer

from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
lemmatizer = FrenchLefffLemmatizer()

lemmas = []
for term in tagged:
    term_l = []
    if lemmatizer.lemmatize(term[0], term[1]) == []:
       term_l = (lemmatizer.lemmatize(term[0]), term[1])
    
    elif type(lemmatizer.lemmatize(term[0], term[1])) == str:
        term_l  = (lemmatizer.lemmatize(term[0], term[1]), term[1])

    else:
        term_l = tuple(lemmatizer.lemmatize(term[0], term[1])[0])

    lemmas.append(term_l)

### **Collocations / Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook  

In [271]:
def extr_ngrams(tagged):
    ngrammes= list(everygrams(tagged, min_len=1, max_len=10))
    print("Avant filtrage, on a {} ngrammes.".format(len(ngrammes)))
    return ngrammes

In [272]:
ngrammes = extr_ngrams(tagged)
#ngrammes_lem = extr_ngrams(tagged_lem)

Avant filtrage, on a 324745 ngrammes.


In [273]:
ngrammes

[(('msss', 'NOM'),),
 (('msss', 'NOM'), ('répertoires', 'NOM')),
 (('msss', 'NOM'), ('répertoires', 'NOM'), ('ressources', 'NOM')),
 (('msss', 'NOM'),
  ('répertoires', 'NOM'),
  ('ressources', 'NOM'),
  ('en', 'PRP')),
 (('msss', 'NOM'),
  ('répertoires', 'NOM'),
  ('ressources', 'NOM'),
  ('en', 'PRP'),
  ('dépendances', 'NOM')),
 (('msss', 'NOM'),
  ('répertoires', 'NOM'),
  ('ressources', 'NOM'),
  ('en', 'PRP'),
  ('dépendances', 'NOM'),
  ('05', 'NUM')),
 (('msss', 'NOM'),
  ('répertoires', 'NOM'),
  ('ressources', 'NOM'),
  ('en', 'PRP'),
  ('dépendances', 'NOM'),
  ('05', 'NUM'),
  ('-', 'PUN')),
 (('msss', 'NOM'),
  ('répertoires', 'NOM'),
  ('ressources', 'NOM'),
  ('en', 'PRP'),
  ('dépendances', 'NOM'),
  ('05', 'NUM'),
  ('-', 'PUN'),
  ('estrie', 'NOM')),
 (('msss', 'NOM'),
  ('répertoires', 'NOM'),
  ('ressources', 'NOM'),
  ('en', 'PRP'),
  ('dépendances', 'NOM'),
  ('05', 'NUM'),
  ('-', 'PUN'),
  ('estrie', 'NOM'),
  ('fiche', 'NOM')),
 (('msss', 'NOM'),
  ('répertoir

### **Extraction des patrons syntaxiques**

*LONG*

In [274]:
def extract_patterns(ngrammes):
    patterns = []
    for ng in ngrammes:
        phrase = tuple([t[0] for t in ng])
        pattern = [t[1] for t in ng]
        patterns.append([phrase, pattern])
    return patterns

In [275]:
phrases = extract_patterns(ngrammes)
# phrases_lem = extract_patterns(ngrammes_lem)

In [276]:
# def freq(phrases):
#     return FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases])

# def freq(phrases):
#     return FreqDist([t[0] for t in phrases])

In [277]:
frequencies = FreqDist(everygrams(tokens, min_len=1, max_len=10))

In [278]:
frequencies

FreqDist({('STOP',): 2426, ('de',): 1503, (':',): 1068, ('et',): 823, ('la',): 805, ('en',): 746, (')',): 718, ('des',): 706, ('(',): 703, ('à',): 513, ...})

### **Filtrage** 
On retire les n-grammes qui débutent ou se terminent par un stopword (antidictionnaire)

In [279]:
# Importer l'antidictionnaire pour filtrer les données

# # Stopwords lemmatisés
# file_path = '../04-filtrage/stopwords_lemmatized.txt'
# with open(file_path, 'r', encoding="utf-8") as f:
#     stopwords_lemmatized = [w.strip('\n').lower() for w in f.readlines()]

# Stopwords fréquents en français (non lemmatisés)
file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]


# Stopwords fréquents en anglais (non lemmatisés)
file_path = '../04-filtrage/stop_words_english.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords += [t.lower().strip('\n') for t in f.readlines()]

*LONG*

In [280]:
def filtrer_stopwords(x): # On s'assure aussi que le premier terme du ngramme est un NOM pour avoir des syntagmes nominaux
    if lng == 'en':
        nom = 'NN'
    if lng == 'fr':
        nom = 'NOM'
    return [term for term in x if not 'STOP' in term[0] and not term[0][0] in stopwords and not term[0][-1] in stopwords \
        and not 'NUM' in term[1] and term[1][0] == nom and not '.' in term[0] and not '-' in term[0] and not ':' in term[0]\
        
        # Une parenthèse fermante peut juste se trouver comme dernier token
        # Si une parenthèse est ouverte, elle doit aussi être fermée (et vice versa)
        and not ')' in term[0][:-1] and not ('(' in term[0] and not ')' in term[0]) \
        and not (')' in term[0] and not '(' in term[0])]

In [281]:
phrases = filtrer_stopwords(phrases)

In [282]:
test = [t[0] for t in phrases]

test

[('msss',),
 ('msss', 'répertoires'),
 ('msss', 'répertoires', 'ressources', 'en', 'dépendances'),
 ('répertoires',),
 ('répertoires', 'ressources', 'en', 'dépendances'),
 ('dépendances',),
 ('estrie',),
 ('estrie', 'fiche'),
 ('estrie', 'fiche', 'de', 'la', 'ressource'),
 ('estrie', 'fiche', 'de', 'la', 'ressource', 'répertoire'),
 ('estrie',
  'fiche',
  'de',
  'la',
  'ressource',
  'répertoire',
  'des',
  'ressources',
  'en',
  'dépendances'),
 ('fiche',),
 ('fiche', 'de', 'la', 'ressource'),
 ('fiche', 'de', 'la', 'ressource', 'répertoire'),
 ('fiche',
  'de',
  'la',
  'ressource',
  'répertoire',
  'des',
  'ressources',
  'en',
  'dépendances'),
 ('ressource',),
 ('ressource', 'répertoire'),
 ('ressource', 'répertoire', 'des', 'ressources', 'en', 'dépendances'),
 ('répertoire',),
 ('répertoire', 'des', 'ressources', 'en', 'dépendances'),
 ('dépendances',),
 ('région',),
 ('région',),
 ('estrie',),
 ('estrie', 'centre'),
 ('estrie', 'centre', "l'"),
 ('estrie', 'centre', "l'"

On retire les n-grammes qui débutent ou se terminent par un token dont la longueur est inférieure à 2 caractères ou supérieure à 18 caractères

In [283]:
def filter_len(x):
    return [term for term in x if \
        (len(term[0][0]) > 2 or term[0][0] == '(')  and (len(term[0][-1]) > 2 or term[0][-1] == ')') and \
        len(term[0][0]) < 18 and len(term[0][-1]) < 18]

In [284]:
phrases = filter_len(phrases)

In [285]:
phrases

[[('msss',), ['NOM']],
 [('msss', 'répertoires'), ['NOM', 'NOM']],
 [('msss', 'répertoires', 'ressources', 'en', 'dépendances'),
  ['NOM', 'NOM', 'NOM', 'PRP', 'NOM']],
 [('répertoires',), ['NOM']],
 [('répertoires', 'ressources', 'en', 'dépendances'),
  ['NOM', 'NOM', 'PRP', 'NOM']],
 [('dépendances',), ['NOM']],
 [('estrie',), ['NOM']],
 [('estrie', 'fiche'), ['NOM', 'NOM']],
 [('estrie', 'fiche', 'de', 'la', 'ressource'),
  ['NOM', 'NOM', 'PRP', 'DET:ART', 'NOM']],
 [('estrie', 'fiche', 'de', 'la', 'ressource', 'répertoire'),
  ['NOM', 'NOM', 'PRP', 'DET:ART', 'NOM', 'NOM']],
 [('estrie',
   'fiche',
   'de',
   'la',
   'ressource',
   'répertoire',
   'des',
   'ressources',
   'en',
   'dépendances'),
  ['NOM',
   'NOM',
   'PRP',
   'DET:ART',
   'NOM',
   'NOM',
   'PRP:det',
   'NOM',
   'PRP',
   'NOM']],
 [('fiche',), ['NOM']],
 [('fiche', 'de', 'la', 'ressource'), ['NOM', 'PRP', 'DET:ART', 'NOM']],
 [('fiche', 'de', 'la', 'ressource', 'répertoire'),
  ['NOM', 'PRP', 'DET:AR

On retire les n-grammes qui apparaissent moins de 3 fois dans le corpus

In [286]:
def filter_freq(x):
    return [term for term in x if frequencies[tuple(term[0])] > 2]

In [287]:
phrases = filter_freq(phrases)

In [288]:
phrases

[[('msss',), ['NOM']],
 [('msss', 'répertoires'), ['NOM', 'NOM']],
 [('msss', 'répertoires', 'ressources', 'en', 'dépendances'),
  ['NOM', 'NOM', 'NOM', 'PRP', 'NOM']],
 [('répertoires',), ['NOM']],
 [('répertoires', 'ressources', 'en', 'dépendances'),
  ['NOM', 'NOM', 'PRP', 'NOM']],
 [('dépendances',), ['NOM']],
 [('estrie',), ['NOM']],
 [('estrie', 'fiche'), ['NOM', 'NOM']],
 [('estrie', 'fiche', 'de', 'la', 'ressource'),
  ['NOM', 'NOM', 'PRP', 'DET:ART', 'NOM']],
 [('estrie', 'fiche', 'de', 'la', 'ressource', 'répertoire'),
  ['NOM', 'NOM', 'PRP', 'DET:ART', 'NOM', 'NOM']],
 [('estrie',
   'fiche',
   'de',
   'la',
   'ressource',
   'répertoire',
   'des',
   'ressources',
   'en',
   'dépendances'),
  ['NOM',
   'NOM',
   'PRP',
   'DET:ART',
   'NOM',
   'NOM',
   'PRP:det',
   'NOM',
   'PRP',
   'NOM']],
 [('fiche',), ['NOM']],
 [('fiche', 'de', 'la', 'ressource'), ['NOM', 'PRP', 'DET:ART', 'NOM']],
 [('fiche', 'de', 'la', 'ressource', 'répertoire'),
  ['NOM', 'PRP', 'DET:AR

In [289]:
#phrases = [[" ".join(term[0]).replace("' ", "'"), " ".join(term[1])] for term in phrases]
phrases = [[term[0], " ".join(term[1])] for term in phrases]
# phrases_lemmatized = filtrer_phrases(phrases_lemmatized, freq_lemmatized)

In [290]:
for phrase in phrases:
    phrase.append(frequencies[tuple(phrase[0])])

In [291]:
print("Après filtrage, on a {} occurrences de ngrammes.".format(len(phrases))) 

Après filtrage, on a 19541 occurrences de ngrammes.


In [292]:
phrases[:15]

[[('msss',), 'NOM', 195],
 [('msss', 'répertoires'), 'NOM NOM', 136],
 [('msss', 'répertoires', 'ressources', 'en', 'dépendances'),
  'NOM NOM NOM PRP NOM',
  136],
 [('répertoires',), 'NOM', 142],
 [('répertoires', 'ressources', 'en', 'dépendances'), 'NOM NOM PRP NOM', 136],
 [('dépendances',), 'NOM', 460],
 [('estrie',), 'NOM', 30],
 [('estrie', 'fiche'), 'NOM NOM', 12],
 [('estrie', 'fiche', 'de', 'la', 'ressource'), 'NOM NOM PRP DET:ART NOM', 12],
 [('estrie', 'fiche', 'de', 'la', 'ressource', 'répertoire'),
  'NOM NOM PRP DET:ART NOM NOM',
  12],
 [('estrie',
   'fiche',
   'de',
   'la',
   'ressource',
   'répertoire',
   'des',
   'ressources',
   'en',
   'dépendances'),
  'NOM NOM PRP DET:ART NOM NOM PRP:det NOM PRP NOM',
  12],
 [('fiche',), 'NOM', 120],
 [('fiche', 'de', 'la', 'ressource'), 'NOM PRP DET:ART NOM', 118],
 [('fiche', 'de', 'la', 'ressource', 'répertoire'),
  'NOM PRP DET:ART NOM NOM',
  118],
 [('fiche',
   'de',
   'la',
   'ressource',
   'répertoire',
   'd

In [293]:
DataFrame(phrases, columns=["Expression", "Patron syntaxique", "Fréquence"])

phrases

[[('msss',), 'NOM', 195],
 [('msss', 'répertoires'), 'NOM NOM', 136],
 [('msss', 'répertoires', 'ressources', 'en', 'dépendances'),
  'NOM NOM NOM PRP NOM',
  136],
 [('répertoires',), 'NOM', 142],
 [('répertoires', 'ressources', 'en', 'dépendances'), 'NOM NOM PRP NOM', 136],
 [('dépendances',), 'NOM', 460],
 [('estrie',), 'NOM', 30],
 [('estrie', 'fiche'), 'NOM NOM', 12],
 [('estrie', 'fiche', 'de', 'la', 'ressource'), 'NOM NOM PRP DET:ART NOM', 12],
 [('estrie', 'fiche', 'de', 'la', 'ressource', 'répertoire'),
  'NOM NOM PRP DET:ART NOM NOM',
  12],
 [('estrie',
   'fiche',
   'de',
   'la',
   'ressource',
   'répertoire',
   'des',
   'ressources',
   'en',
   'dépendances'),
  'NOM NOM PRP DET:ART NOM NOM PRP:det NOM PRP NOM',
  12],
 [('fiche',), 'NOM', 120],
 [('fiche', 'de', 'la', 'ressource'), 'NOM PRP DET:ART NOM', 118],
 [('fiche', 'de', 'la', 'ressource', 'répertoire'),
  'NOM PRP DET:ART NOM NOM',
  118],
 [('fiche',
   'de',
   'la',
   'ressource',
   'répertoire',
   'd

In [294]:
print("Après filtrage, on a {} ngrammes uniques.".format(len(DataFrame(phrases).drop_duplicates())))

Après filtrage, on a 2120 ngrammes uniques.


### **Filtrage (Patrons syntaxiques)**  
Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

On veut aller extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessus ; voir le Notebook *Mesh_extract.ipynb*). Pour ce faire, nous allons donc ne sélectionner que les ngrammes qui y correspondent. 

In [295]:
file_patterns = '../04-filtrage/MeSH/mesh_patterns-fr.csv'

with open (file_patterns, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist() #[:200] # On prend les 200 structures syntaxiques les plus fréquentes dans les MeSH

In [296]:
def filter_patterns(phrases):
    return [t for t in phrases if t[1] in patterns and not 'NUM' in t[1]] # and not 'NOM NOM' in t[1]
# terms_lemmatized = [t for t in phrases_lemmatized if t[1] in patterns]

In [297]:
terms = filter_patterns(phrases)
#terms = phrases

In [298]:
terms

[[('msss',), 'NOM', 195],
 [('msss', 'répertoires'), 'NOM NOM', 136],
 [('répertoires',), 'NOM', 142],
 [('répertoires', 'ressources', 'en', 'dépendances'), 'NOM NOM PRP NOM', 136],
 [('dépendances',), 'NOM', 460],
 [('estrie',), 'NOM', 30],
 [('estrie', 'fiche'), 'NOM NOM', 12],
 [('estrie', 'fiche', 'de', 'la', 'ressource'), 'NOM NOM PRP DET:ART NOM', 12],
 [('estrie', 'fiche', 'de', 'la', 'ressource', 'répertoire'),
  'NOM NOM PRP DET:ART NOM NOM',
  12],
 [('fiche',), 'NOM', 120],
 [('fiche', 'de', 'la', 'ressource'), 'NOM PRP DET:ART NOM', 118],
 [('fiche', 'de', 'la', 'ressource', 'répertoire'),
  'NOM PRP DET:ART NOM NOM',
  118],
 [('ressource',), 'NOM', 332],
 [('ressource', 'répertoire'), 'NOM NOM', 118],
 [('ressource', 'répertoire', 'des', 'ressources', 'en', 'dépendances'),
  'NOM NOM PRP:det NOM PRP NOM',
  118],
 [('répertoire',), 'NOM', 166],
 [('répertoire', 'des', 'ressources', 'en', 'dépendances'),
  'NOM PRP:det NOM PRP NOM',
  140],
 [('dépendances',), 'NOM', 460],

In [299]:
print("Le filtrage syntaxique élimine environ {} % des termes".format(round((len(phrases) - len(terms)) / len(phrases) * 100)))
print("On avait {} ngrammes, ".format(len(phrases)) + "on en a maintenant {}.".format(len(terms)))

Le filtrage syntaxique élimine environ 24 % des termes
On avait 19541 ngrammes, on en a maintenant 14810.


In [300]:
# import pandas as pd

# def extract_terms(liste_terms):
#     file_path = '../04-filtrage/output/'
#     tab = pd.DataFrame(terms, columns= ["Expression", "Structure syntaxique", "Fréquence"]).drop_duplicates(subset='Expression', keep="last")
#     tab.sort_values(["Fréquence"], 
#                         axis=0,
#                         ascending=[False], 
#                         inplace=True)

#     return 

#     # file_path = path.join(file_path, tag, tag)                    
#     # tab.to_csv(file_path + '_terms.csv')

# extract_terms(terms)
# #extract_terms(terms_lemmatized)

In [301]:
for phrase in terms:
    phrase[0] = tuple(phrase[0])


terms_patterns = DataFrame(terms, columns = ["Expression", "Structure syntaxique", "Fréquence"])
terms_patterns = terms_patterns.to_dict('records')
dict_patterns = {}
for term in terms_patterns:
     exp = term['Expression']
     pattern = term['Structure syntaxique']
     dict_patterns[exp] = pattern

In [302]:
terms_patterns

[{'Expression': ('msss',), 'Structure syntaxique': 'NOM', 'Fréquence': 195},
 {'Expression': ('msss', 'répertoires'),
  'Structure syntaxique': 'NOM NOM',
  'Fréquence': 136},
 {'Expression': ('répertoires',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 142},
 {'Expression': ('répertoires', 'ressources', 'en', 'dépendances'),
  'Structure syntaxique': 'NOM NOM PRP NOM',
  'Fréquence': 136},
 {'Expression': ('dépendances',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 460},
 {'Expression': ('estrie',), 'Structure syntaxique': 'NOM', 'Fréquence': 30},
 {'Expression': ('estrie', 'fiche'),
  'Structure syntaxique': 'NOM NOM',
  'Fréquence': 12},
 {'Expression': ('estrie', 'fiche', 'de', 'la', 'ressource'),
  'Structure syntaxique': 'NOM NOM PRP DET:ART NOM',
  'Fréquence': 12},
 {'Expression': ('estrie', 'fiche', 'de', 'la', 'ressource', 'répertoire'),
  'Structure syntaxique': 'NOM NOM PRP DET:ART NOM NOM',
  'Fréquence': 12},
 {'Expression': ('fiche',), 'Structure syntaxique': 'N

### **Filtrage (Collocations statistiquement significatives)** Log-Likelihood Ratio

[Notebook - Collocation extraction methodologies compared](https://notebooks.githubusercontent.com/view/ipynb?azure_maps_enabled=false&browser=chrome&color_mode=auto&commit=33868e847376764d7733cd958986c88dedfaec97&device=unknown&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f746f64642d636f6f6b2f4d4c2d596f752d43616e2d5573652f333338363865383437333736373634643737333363643935383938366338386465646661656339372f70726f626162696c69737469635f6c616e67756167655f6d6f64656c696e672f636f6c6c6f636174696f6e5f65787472616374696f6e732e6970796e62&enterprise_enabled=false&logged_in=false&nwo=todd-cook%2FML-You-Can-Use&path=probabilistic_language_modeling%2Fcollocation_extractions.ipynb&platform=android&repository_id=167140788&repository_type=Repository&version=102)

On applique un test d'hypothèse statistique aux n-grammes sur lesquels une probabilité a été mesurée (Log-likelihood ratio) - seuls les n-grammes dont le test est significatif seront conservés.
On considère que l'apparition de ces collocations dans notre corpus n'est pas dûe au hasard.

In [303]:
def loglikelihood_ratio(c_prior, c_n, c_ngram, N):
    """
    Compute the ratio of two hypotheses of likelihood and return the ratio.
    The formula here and test verification values are taken from 
    Manning & Schūtze _Foundations of Statistical Natural Language Processing_ p.172-175
    Parameters:
    c_prior: count of word 1 if bigrams or count of [w1w2 .. w(n-1)] if ngram
    c_n : count of word 2 if bigrams or count of wn if ngram
    c12: count of bigram (w1, w2) if bigram or count of ngram if ngram
    N: the number of words in the corpus
    """

    p = c_n / N
    p1 = c_ngram / c_prior
    p2 = (c_n - c_ngram) / (N - c_prior)   
    # We proactively trap a runtimeWarning: divide by zero encountered in log,
    # which may occur with extreme collocations
    import warnings
    with warnings.catch_warnings(): # this will reset our filterwarnings setting
        warnings.filterwarnings('error')
        try:
            return (np.log(binom.pmf(c_ngram, c_prior, p)) 
                    + np.log(binom.pmf(c_n - c_ngram, N - c_prior, p)) 
                    - np.log(binom.pmf(c_ngram, c_prior, p1) )
                    - np.log(binom.pmf(c_n - c_ngram, N - c_prior, p2)))             
        except Warning:
            return np.inf 

In [304]:
len_prior = len(terms)
print("Au départ, on a {} ngrammes.".format(len_prior))

Au départ, on a 14810 ngrammes.


In [305]:
frequencies

FreqDist({('STOP',): 2426, ('de',): 1503, (':',): 1068, ('et',): 823, ('la',): 805, ('en',): 746, (')',): 718, ('des',): 706, ('(',): 703, ('à',): 513, ...})

In [306]:
# Pour le calcul des probabilités, on a besoin de traiter séparément les ngrammes selon la valeur de n
N = len(tokens)
fd_tokens = nltk.FreqDist(tokens)

In [307]:
def llr_ngrammes(terms):
    llr = []

    ngrammes = [term[0] for term in terms]
        
    for t in ngrammes:
        if len(t) == 1:
            try:
                llr.append({'Terme' : str(t[0]), 'Structure syntaxique': dict_patterns[t], 'Fréquence (TF)' : fd_tokens[str(t[0])], 'LLR': '-', 'p-value': '-'})
            except Exception as e:
                print(t, str(e))
        else:
            c_prior = frequencies[t[:-1]] # Antécédent = P(w1w2..w_n-1) (si on considère que P(w1w2...wn) = P(wn) | P(w1w2...w_n-1)
            c_n = fd_tokens[t[-1]]     # Dernier mot du ngramme  P(wn)
            c_ngram = frequencies[t]             # Le ngramme lui-même P(w1w2w3..wn)

            try:
                res = -2 * loglikelihood_ratio(c_prior, c_n, c_ngram, N)
                p_value = chi2.sf(res, 1) # 1 degrees of freedom
                #if res == float('-inf') :
                #    res = 50000

                if p_value < 0.01 or (res == float('-inf')):
                    #llr.append({'Collocation' : " ".join(t).replace("' ", "'").replace("( ", "(").replace(" )", ")"), 'Structure syntaxique': dict_patterns[" ".join(t).replace("' ", "'")], 'Fréquence' : c_ngram, 'LLR': res, 'p-value': p_value})
                    llr.append({'Terme' : t, 'Structure syntaxique': dict_patterns[t], 'Fréquence (TF)' : c_ngram, 'LLR': res, 'p-value': p_value})
            
            except Exception as e:
                print(t, str(e))
            
    return llr


In [308]:
significant_coll = llr_ngrammes(terms)
#terms = [{'Collocation' : t[0], 'Structure syntaxique': t[1], 'Fréquence' : t[2]} for t in terms] # Si on veut pas appliquer le LLR

In [309]:
significant_coll


[{'Terme': 'msss',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 195,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': ('msss', 'répertoires'),
  'Structure syntaxique': 'NOM NOM',
  'Fréquence (TF)': 136,
  'LLR': 1472.0505184618928,
  'p-value': 0.0},
 {'Terme': 'répertoires',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 142,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': ('répertoires', 'ressources', 'en', 'dépendances'),
  'Structure syntaxique': 'NOM NOM PRP NOM',
  'Fréquence (TF)': 136,
  'LLR': 1202.2540852500586,
  'p-value': 1.9743489378862806e-263},
 {'Terme': 'dépendances',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 460,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'estrie',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 30,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': ('estrie', 'fiche'),
  'Structure syntaxique': 'NOM NOM',
  'Fréquence (TF)': 12,
  'LLR': 95.39755402656563,
  'p-value': 1.557459912218746e-22},
 {'Terme': ('estrie', 'fiche', 'de', 'la

In [310]:
print('Après avoir calculé le log-likelihood ratio, on a retiré {} collocations qui n\'étaient pas statistiquement significatives.'.format(len(terms) - len(significant_coll)))
print('Ça représente environ {} % de nos n-grammes.'.format(round((len(terms) - len(significant_coll)) / len(terms) *100 )))

Après avoir calculé le log-likelihood ratio, on a retiré 15 collocations qui n'étaient pas statistiquement significatives.
Ça représente environ 0 % de nos n-grammes.


In [311]:
df = DataFrame(significant_coll).sort_values(by = "Fréquence (TF)", ascending=False).drop_duplicates()

# On veut faire join pour tous les termes[collocation]
def join_term(x):
    if type(x) == tuple:
        return " ".join(x).replace("' ", "'").replace("( ", "(").replace(" )", ")")
    else:
        return x

df['Terme'] = df['Terme'].apply(lambda x: join_term(x))

if lng == 'en':
    acteur = acteur + '_' + lng
if tag:
    output_path = path.join('../04-filtrage/output/', acteur + '_' + tag + '_significant-collocations.csv') 
else:
    output_path = path.join('../04-filtrage/output/', acteur + '_significant-collocations.csv') 

df.to_csv(output_path)

df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value
1353,dépendances,NOM,460,-,-
3386,ressource,NOM,332,-,-
1354,région,NOM,247,-,-
12204,québec,NOM,204,-,-
3027,msss,NOM,195,-,-
...,...,...,...,...,...
6805,entente,NOM,3,-,-
6794,négociations,NOM,3,-,-
6790,projets diffusé par santé canada,NOM VER:pper PRP NOM NOM,3,45.51794,0.0
11686,lien,NOM,3,-,-


### **Filtrage - Fréquence documentaire**
** Il y aurait quelque chose à modifier ici ; en tokenisant, on supprimer les frontières entre les mots qui sont des tirets ou autres caractères qu'un espace ou un apostrophe ; ça fait en sorte qu'on a une fréquence documentaire de 0 pour les ngrammes qui n'ont plus la forme exacte qu'ils avaient dans le corpus orginal. **

*LONG*  
Est-ce qu'il y a moyen de rendre ça plus efficace ?

In [312]:
dfs = {term: len([doc for doc in text if term in doc]) for term in df['Terme'].tolist()}

In [313]:
type(dfs)

dict

In [314]:
max_df = round(0.85 * nb_docs)        # Pour rejeter les termes qui se retrouvent dans plus de 85% des documents 
max_df

140

In [315]:
max_df_observed = dfs[max(dfs, key=dfs.get)] # Voir quelle est la fréquence documentaire maximale qu'on retrouve
max_df_observed

164

In [316]:
dfs = {term:df for term,df in dfs.items() if df < max_df and df > 0} # Si df = 0, c'est un artefact créé par le prétraitement lui-même

Porter attention aux termes qui sortent ici - en comptant la fréquence documentaire, techniquement on ne devrait pas retrouver de 0

In [317]:
zeros =  {term:df for term,df in dfs.items() if df == 0}  
print(len(zeros))
zeros 

0


{}

In [318]:
dfs = DataFrame(list(dfs.items()),columns = ['Terme','Fréquence documentaire (DF)']) 

dfs

,Terme,Fréquence documentaire (DF)
0,québec,138
1,services,64
2,programme,106
3,santé,34
4,répertoire des ressources en dépendances,138
...,...,...
1239,entente,9
1240,négociations,3
1241,projets diffusé par santé canada,3
1242,lien,119


In [319]:
df = df.merge(dfs, on='Terme').drop_duplicates()

df.sort_values(['Fréquence (TF)'], 
            axis=0,
            ascending=[False], 
            inplace=True)
if lng == 'en':
    acteur = acteur + '_' + lng
if tag:
    output_path = path.join('../04-filtrage/output/', acteur + '_' + tag + '_significant-collocations.csv') 
else:
    output_path = path.join('../04-filtrage/output/', acteur + '_significant-collocations.csv') 
df.to_csv(output_path)

In [320]:
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF)
0,québec,NOM,204,-,-,138
1,services,NOM,170,-,-,64
2,programme,NOM,143,-,-,106
3,santé,NOM,142,-,-,34
4,répertoire des ressources en dépendances,NOM PRP:det NOM PRP NOM,140,1239.130453,0.0,138
...,...,...,...,...,...,...
949,projets financés par le puds,NOM VER:pper PRP DET:ART NOM,3,48.2421,0.0,3
948,connaissances entourant,NOM VER:ppre,3,50.280888,0.0,3
947,cohérence avec les orientations,NOM PRP DET:ART NOM,3,52.17743,0.0,3
946,cohérence avec les orientations du québec,NOM PRP DET:ART NOM PRP:det NOM,3,30.465432,0.0,3


In [321]:
list_terms = df['Terme'].tolist()

### **KWIC (Keyword in Context)**
Termes d'intérêt : 
- « Programme »
- « Plan »
- « Service(s) de » 
- « Intervenant(e) en »
- « Professionnel de »
- « Institut (du/de) »
- « Groupe de recherche en »
- « Personne »
- « Infirmière (en) »

In [322]:
# Dans notre cas on veut que ça débute par le mot-clé donc le contexte est un peu plus simple
# penser à généraliser avec des expressions régulières
kw = ['programme', 'plan ', 'service', 'intervenant', 'infirmière en', 'institut', 'groupe de recherche', 'personne', 'maladie']

ngrammes_kwic = [" ".join([t[0] for t in ng]).replace("' ", "'") for ng in ngrammes]

In [323]:
extrant = DataFrame(columns=['Mot-clé','Concordance', 'Fréquence (TF)'])
kwic = {w : [] for w in kw} 

In [324]:
for t in ngrammes_kwic: # on pourrait aussi chercher dans les terms, mais on perd certains termes d'intérêt avec le filtrage syntaxique
    for w in kw:
        if t.startswith(w):
            kwic[w].append(t)

In [325]:
kwic = {term: FreqDist(kwic[term]) for term in kwic}

In [326]:
for term in kw:
    df = DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence"])
    df.sort_values(["Fréquence"], 
        axis=0,
        ascending=[False], 
        inplace=True)

    df.insert(0, 'Mot-clé', term)
    extrant = concat([extrant, df])


extrant = extrant[extrant['Fréquence'] > 30] 

file_path = '../04-filtrage/output/'
file_path = path.join(file_path, tag, tag)


#extrant.to_csv(file_path + '_KWIC' +'.csv')

extrant

,Mot-clé,Concordance,Fréquence (TF),Fréquence
0,programme,programme,NaN,143.0
1,programme,programme :,NaN,84.0
0,service,services,NaN,170.0
75,service,services sociaux,NaN,88.0
28,service,service,NaN,60.0
10,service,services de,NaN,50.0
12,service,services de réadaptation en,NaN,36.0
11,service,services de réadaptation,NaN,36.0
13,service,services de réadaptation en externe,NaN,36.0
0,personne,personnes,NaN,120.0


### **Extraction de termes MeSH**

In [327]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/MeSH/mesh-fr.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [328]:
extr_mesh = tokenizer_mesh.tokenize(list_terms)

In [329]:
# MODIFICATION À APPORTER : dans l'extrant où on voit les termes mesh, ajouter la fréquence

In [330]:
termes_mesh = []

for t in extr_mesh:
    if t in mesh:
        termes_mesh.append(t)

In [331]:
# PT ajouter plutôt, dans le dataframe résultant, une colonne isMeSH et isSNOMED - le faire en une seule boucle plutôt que 2 ?

In [332]:
file_path = '../04-filtrage/output/'
if tag:
    file_path = file_path + acteur + '_' + tag

else :
    file_path = file_path + acteur

df = DataFrame(termes_mesh)
df.to_csv(file_path + '_MeSH.csv')

### **Extraction de termes SNOMED**

In [333]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/SNOMED_fr.csv'

with open(file_path, 'r', encoding='utf-8') as f:
    sm = read_csv(f, sep=';')
    sm = list(dict.fromkeys([str(t).strip().lower() for t in sm['term'].tolist()]))

    sm = [tuple(tokenizer_re.tokenize(w)) for w in sm if len(w.split()) > 1]
    tokenizer_sm = MWETokenizer(sm, separator = ' ')

    sm = [tokenizer_sm.tokenize(w)[0].lower() for w in sm]

In [334]:
extr_sm = tokenizer_sm.tokenize([t for t in candidate_terms])

NameError: name 'candidate_terms' is not defined

In [ ]:
termes_sm = []

for t in extr_sm:
    if t in sm:
        termes_sm.append(t)

In [ ]:
file_path = '../04-filtrage/output/' 
if tag:
    file_path = file_path + acteur + '_' + tag 

else :
    file_path = file_path + acteur

df = DataFrame(termes_sm)

df.to_csv(file_path + '_SNOMED.csv')